In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install transformers googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.6 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=1edf791fb7db5ea66c4b79e94854e222c83b174ee9572c9538f0325f50e655e7
  Stored in directory: /root/.cache/pip/wheels/39/17/6f/66a045ea3d168826074691b4b787b8f324d3f646d755443fda
Successfully built googletrans
  Attempting uninstall: hyperframe
    Found existing installation: hyperframe 6.1.0
    Uninstalling hype

In [3]:
from transformers import MarianMTModel, MarianTokenizer
from googletrans import Translator

2025-04-13 23:27:48.639430: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744586868.822716      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744586868.875585      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# Load translation model
def load_translation_model():
    model_name = "Helsinki-NLP/opus-mt-en-fr"
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)
    return tokenizer, model

tokenizer, model = load_translation_model()
translator = Translator()

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [14]:
import torch

def translate_batch_to_french(text):
    """
    Translates the given text into French.

    Parameters:
        text (str or list of str): The text(s) to be translated.

    Returns:
        str: The French translation, joined as a single string.
    """
    # Use GPU if available
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    # Ensure texts is a list
    if isinstance(text, str):
        texts = [text]
    else:
        texts = text
    
    # Tokenize the input texts
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    inputs = {key: val.to(device) for key, val in inputs.items()}
    
    # Move the model to the appropriate device
    model.to(device)
    
    # Generate translations
    outputs = model.generate(**inputs)
    
    # Decode the generated tokens into strings
    translations = [tokenizer.decode(t, skip_special_tokens=True) for t in outputs]
    
    # Join multiple translations into one string if necessary
    return " ".join(translations)


In [15]:
def chatbot():
    print("Bonjour! I am your French Teaching Chatbot. Ask me anything!")
    print("Examples: \n- 'Translate I love apples'\n- 'Teach me vocabulary'\n- 'Explain French grammar'\nType 'exit' to quit.")
    
    while True:
        user_input = input("You: ").strip()
        if user_input.lower() == "exit":
            print("Au revoir! Have a great day learning French!")
            break
        
        if "translate" in user_input.lower():
            start = user_input.lower().find("translate") + len("translate")
            english_text = user_input[start:].strip()
            if not english_text:
                english_text = user_input
            french_translation = translate_batch_to_french(english_text)
            print("Bot: Translation: " + french_translation)
        
        elif "vocabulary" in user_input.lower():
            print("Bot: Here are some basic French words:")
            print("1. Bonjour - Hello")
            print("2. Merci - Thank you")
            print("3. Oui - Yes")
            print("4. Non - No")
            print("5. Pomme - Apple")
        
        elif "grammar" in user_input.lower():
            print("Bot: Basic French Grammar:")
            print("- In French, adjectives agree with the noun in gender and number.")
            print("- Example: 'une pomme verte' (a green apple) vs. 'un chat noir' (a black cat).")
        
        else:
            french_translation = translate_batch_to_french(user_input)
            print("Bot: Translation: " + french_translation)

if __name__ == "__main__":
    chatbot()


Bonjour! I am your French Teaching Chatbot. Ask me anything!
Examples: 
- 'Translate I love apples'
- 'Teach me vocabulary'
- 'Explain French grammar'
Type 'exit' to quit.


You:  good morning


Bot: Translation: Bonjour.


You:  my name is Rasheed


Bot: Translation: Mon nom est Rasheed


You:  I am a student of UM6P


Bot: Translation: Je suis un étudiant de UM6P


You:  EXIT


Au revoir! Have a great day learning French!
